In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import numpy as np
data = pd.read_csv(r"..\Data\ADS_with_all_dummies.csv")
data = data.drop(['Unnamed: 0', 'finalspend_usd', "effectiveness_usd", "starttime"], axis = 1)

# Load the model
from catboost import CatBoostRegressor
loaded_model = CatBoostRegressor()


# MENTION THE ZONE
zone = "zone_APAC"
category = "categ_l1_COMMERCIAL"

loaded_model.load_model(f"model_Arun_{zone}_{category}.cbm")
data = data[data[zone]==1][data[category] == 1]


C:\Users\40107607\AppData\Local\Temp\ipykernel_21152\1470681060.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data[data[zone]==1][data[category] == 1]


In [5]:
data = data[loaded_model.feature_names_ + ["data","event_id", "efficiency"]]

In [6]:
train = data[data["data"]== 'train']
forecast = data[data['data'] == 'submission_df']

In [7]:
forecast

,reviewtimeperiod,biddingperiod,overtimeperiod(minutes),bidrankthattriggersovertime,canparticipantsplacebidsduringpreviewperiod_Do not allow prebids,improvebidamountby_Percentage,"showlineitemlevelrankinlot_Yes, to Buyers and Participants",enabletrafficlightbidding_Yes,setareviewperiodafterlotoritemcloses_Yes,setareviewperiodafterlotcloses_Yes,...,categ_l2_IND CAPEX,event_template_ABI Reverse Auction - NI,bestbid_usd,Count_supplier,itemno,item_name,zone_APAC,data,event_id,efficiency
15020,1.0,0.0,2.0,2.0,0,1,1,0,1,0,...,0,0,9652.92,739364,1.0,1,1,submission_df,ACOU,0.0
15021,2.0,30.0,3.0,3.0,0,0,0,1,1,0,...,0,1,122529.53,739364,1.0,1,1,submission_df,AGKB,0.0
15047,2.0,30.0,3.0,2.0,0,1,0,1,1,0,...,0,1,94936.13,739364,1.0,1,1,submission_df,EQNM,0.0
15053,2.0,30.0,3.0,4.0,0,0,1,0,1,0,...,0,1,28461.14,739364,1.0,1,1,submission_df,CHES,0.0
15057,2.0,30.0,2.0,2.0,0,1,1,0,1,0,...,0,1,200426.79,739364,1.0,1,1,submission_df,DGXX,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19676,2.0,30.0,2.0,2.0,0,1,1,0,1,0,...,0,1,96779.37,739364,1.0,1,1,submission_df,CFOI,0.0
19686,2.0,30.0,5.0,4.0,0,1,0,1,1,0,...,0,0,13911.56,739364,1.0,1,1,submission_df,FHVG,0.0
19698,2.0,30.0,3.0,2.0,0,1,0,1,1,0,...,0,0,92571.09,739364,1.0,1,1,submission_df,ANFX,0.0
19702,2.0,20.0,5.0,4.0,0,1,1,1,1,0,...,0,1,489619.71,739364,2.0,2,1,submission_df,EMGH,0.0


In [8]:
binary_variables = [
    'canparticipantsplacebidsduringpreviewperiod_Do not allow prebids',
    'improvebidamountby_Percentage',
    'showlineitemlevelrankinlot_Yes, to Buyers and Participants',
    'enabletrafficlightbidding_Yes',
    'setareviewperiodafterlotoritemcloses_Yes',
    'setareviewperiodafterlotcloses_Yes',
    'canparticipantsseeranks?_Their own rank',
    'canparticipantsseeranks?_No',
]

continuous_variables = [
    'reviewtimeperiod',
    'biddingperiod', 
    'overtimeperiod(minutes)',
    'bidrankthattriggersovertime',
]

other_variables = [
    'baselinespend_usd',
    # 'zone_APAC',
    'Count_supplier',
    'event_template_ABI Reverse Auction - NI',
    'categ_l2_IND CAPEX',
    'item_name',
    'Mean_Lot_Item_Bid_Version',
    'ratio',
    'itemno',
    'bestbid_usd',
    'participant',
    'Count_Item_Name'
]
X_forecast = forecast.drop(['data', 'event_id'], axis = 1)
X_forecast

,reviewtimeperiod,biddingperiod,overtimeperiod(minutes),bidrankthattriggersovertime,canparticipantsplacebidsduringpreviewperiod_Do not allow prebids,improvebidamountby_Percentage,"showlineitemlevelrankinlot_Yes, to Buyers and Participants",enabletrafficlightbidding_Yes,setareviewperiodafterlotoritemcloses_Yes,setareviewperiodafterlotcloses_Yes,...,baselinespend_usd,participant,categ_l2_IND CAPEX,event_template_ABI Reverse Auction - NI,bestbid_usd,Count_supplier,itemno,item_name,zone_APAC,efficiency
15020,1.0,0.0,2.0,2.0,0,1,1,0,1,0,...,11356.38,4.0,0,0,9652.92,739364,1.0,1,1,0.0
15021,2.0,30.0,3.0,3.0,0,0,0,1,1,0,...,120000.00,3.0,0,1,122529.53,739364,1.0,1,1,0.0
15047,2.0,30.0,3.0,2.0,0,1,0,1,1,0,...,87529.39,4.0,0,1,94936.13,739364,1.0,1,1,0.0
15053,2.0,30.0,3.0,4.0,0,0,1,0,1,0,...,28500.00,4.0,0,1,28461.14,739364,1.0,1,1,0.0
15057,2.0,30.0,2.0,2.0,0,1,1,0,1,0,...,202500.00,4.0,0,1,200426.79,739364,1.0,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19676,2.0,30.0,2.0,2.0,0,1,1,0,1,0,...,97500.00,4.0,0,1,96779.37,739364,1.0,1,1,0.0
19686,2.0,30.0,5.0,4.0,0,1,0,1,1,0,...,21293.20,4.0,0,0,13911.56,739364,1.0,1,1,0.0
19698,2.0,30.0,3.0,2.0,0,1,0,1,1,0,...,95486.61,3.0,0,0,92571.09,739364,1.0,1,1,0.0
19702,2.0,20.0,5.0,4.0,0,1,1,1,1,0,...,503014.24,8.0,0,1,489619.71,739364,2.0,2,1,0.0


In [9]:
from scipy.optimize import minimize
import numpy as np

def objective_function_scipy(vars, other_values, loaded_model, continuous_count, binary_count):
    """
    Objective function for optimization.
    """
    continuous_vars = vars[:continuous_count]
    binary_vars = vars[continuous_count:]
    # binary_vars = np.round(vars[continuous_count:])  # Ensure binary variables are rounded to 0 or 1
    candidate = list(continuous_vars) + list(binary_vars) + other_values
    # print(candidate)
    # print(loaded_model.predict(candidate))
    return -loaded_model.predict(candidate)  # Negate because we maximize in Pyomo but SciPy minimizes

def scipy_optimization(x_row):
    """
    Optimization using SciPy.
    """
    # Extract problem data
    other_values = list(x_row[other_variables])  # Fixed values for other variables
    continuous_count = len(continuous_variables)
    binary_count = len(binary_variables)
    
    # Define bounds for variables
    bounds = [(0, 10), (0,100), (0,10), (0,5)] + [(0, 1)] * binary_count  # Continuous [0, 120], Binary [0, 1]
    
    # Define initial guesses
    x0 = list(x_row[continuous_variables + binary_variables])
    # x0 = [0] * len(continuous_variables + binary_variables)
    # x0 = list(x_row[continuous_variables]) + [0.5] * binary_count

    # Define constraints
    constraints = [
        {'type': 'ineq', 'fun': lambda vars: 10 - vars[0]},  # y[0] <= 10
        {'type': 'ineq', 'fun': lambda vars: 100 - vars[1]},  # y[1] <= 100
        {'type': 'ineq', 'fun': lambda vars: 10 - vars[2]},  # y[2] <= 10
        {'type': 'ineq', 'fun': lambda vars: 5 - vars[3]},  # y[3] <= 5
        {'type': 'ineq', 'fun': lambda vars: 1 - vars[continuous_count + 6] - vars[continuous_count + 7]},  # y[3] <= 5
    ]
    constraints = constraints + [{'type' : 'ineq', 'fun' : lambda vars: 1 - vars[continuous_count + i]} for i in range(binary_count)]

    # Solve using SciPy
    result = minimize(
        objective_function_scipy,
        x0=x0,
        args=(other_values, loaded_model, continuous_count, binary_count),
        bounds=bounds,
        constraints=constraints,
        # method='SLSQP',  # Sequential Least Squares Programming
        method = "COBYLA",
        options={'disp': True}
    )

    # Extract results
    optimized_vars = result.x
    continuous_vars = np.round(optimized_vars[:continuous_count])
    binary_vars = np.round(optimized_vars[continuous_count:])  # Round binary variables

    # Display results
    print("Optimization Result:")
    print("Continuous Variables:", continuous_vars)
    print("Binary Variables:", binary_vars)
    print("initial vars: ", list(x0))

    best_candidate = list(continuous_vars) + list(binary_vars) + other_values
    result_objective = loaded_model.predict(best_candidate) 
    print("Objective Value:", result_objective)

    return result_objective, list(continuous_vars) + list(binary_vars)

In [10]:
optimized_effienciencies = []
optimized_x = []
for i in range(X_forecast.shape[0]):
# for i in range(10):
    x_row = X_forecast.iloc[i]
    print(X_forecast.shape[0] - i)
    best_eff, best_x = scipy_optimization(x_row)
    optimized_effienciencies.append(best_eff)
    optimized_x.append(best_x)

731
Optimization Result:
Continuous Variables: [1. 0. 2. 4.]
Binary Variables: [-0.  1.  1.  1.  1.  0.  1. -0.]
initial vars:  [1.0, 0.0, 2.0, 2.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.12393434308473095
730
Optimization Result:
Continuous Variables: [ 5. 29.  5.  4.]
Binary Variables: [ 1.  1.  1.  1.  1.  1.  1. -0.]
initial vars:  [2.0, 30.0, 3.0, 3.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.1433981150611708
729
Optimization Result:
Continuous Variables: [ 3. 30.  4.  3.]
Binary Variables: [ 1.  1. -0.  1.  1. -0.  1.  0.]
initial vars:  [2.0, 30.0, 3.0, 2.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Objective Value: 0.11518038619029508
728
Optimization Result:
Continuous Variables: [ 2. 30.  6.  5.]
Binary Variables: [ 1.  1.  1.  0.  1. -0.  1. -0.]
initial vars:  [2.0, 30.0, 3.0, 4.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
Objective Value: 0.11707202335445102
727
Optimization Result:
Continuous Variables: [ 2. 30.  3.  2.]
Binary Variables:

In [11]:
y_pred = loaded_model.predict(X_forecast)
y_pred

array([ 0.09822201,  0.05327277,  0.09955495,  0.1077072 ,  0.43616119,
        0.16405525,  0.12663922,  0.07805035,  0.08736823,  0.08247337,
        0.06480034,  0.10076405,  0.12921601,  0.08478427,  0.09896833,
        0.17736146,  0.11120868,  0.10710463,  0.10480092,  0.08885489,
        0.17054172,  0.10121296,  0.2239486 ,  0.13115613,  0.17771685,
        0.06576236,  0.13144858,  0.15107804,  0.09771109,  0.11320682,
        0.11298404,  0.14202094,  0.08469008,  0.14401326,  0.06818343,
        0.10387664,  0.05241407,  0.14563268,  0.09189718,  0.06837497,
        0.12989733,  0.061852  ,  0.06487836,  0.08580734,  0.05474321,
        0.0916502 ,  0.0829452 ,  0.11796633,  0.11313376,  0.07163139,
        0.08223205,  0.05645825,  0.07133515,  0.08443801,  0.15331118,
        0.3515297 ,  0.09851357,  0.12135515,  0.08164237,  0.0991472 ,
        0.06866754,  0.06382316,  0.13297641,  0.07640759,  0.21253249,
        0.04304631,  0.05768345,  0.12069669,  0.12697301,  0.07

In [15]:
final = pd.DataFrame(optimized_x, columns = loaded_model.feature_names_[:len(continuous_variables) + len(binary_variables)])
final_1 = pd.concat([final, forecast[other_variables + ['event_id']].reset_index(drop = True)], axis = 1)
# final_1['event_id'] = forecast['event_id']
final_1['Optimized_efficiency'] = optimized_effienciencies
final_1['Initial_efficiency'] = y_pred
final_1['Uplift'] = final_1['Optimized_efficiency'] - final_1['Initial_efficiency']
final_1['final_spend'] = (1-final_1['Initial_efficiency'])** final_1['bestbid_usd']
final_1['final_spend_optimized'] = (1-final_1['Optimized_efficiency'])** final_1['bestbid_usd']
final_1['Uplift_dollars'] = final_1['Uplift'] * final_1['bestbid_usd']

In [16]:
print(f"Uplift in effiency : {sum(final_1['Uplift_dollars']/sum(final_1['bestbid_usd']))*100} %")

Uplift in effiency : 4.521123256843725 %


In [17]:
final_1

,reviewtimeperiod,biddingperiod,overtimeperiod(minutes),bidrankthattriggersovertime,canparticipantsplacebidsduringpreviewperiod_Do not allow prebids,improvebidamountby_Percentage,"showlineitemlevelrankinlot_Yes, to Buyers and Participants",enabletrafficlightbidding_Yes,setareviewperiodafterlotoritemcloses_Yes,setareviewperiodafterlotcloses_Yes,...,bestbid_usd,participant,Count_Item_Name,event_id,Optimized_efficiency,Initial_efficiency,Uplift,final_spend,final_spend_optimized,Uplift_dollars
0,1.0,0.0,2.0,4.0,-0.0,1.0,1.0,1.0,1.0,0.0,...,9652.92,4.0,739364,ACOU,0.123934,0.098222,0.025712,0.0,0.0,248.199125
1,5.0,29.0,5.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,...,122529.53,3.0,739364,AGKB,0.143398,0.053273,0.090125,0.0,0.0,11043.016596
2,3.0,30.0,4.0,3.0,1.0,1.0,-0.0,1.0,1.0,-0.0,...,94936.13,4.0,739364,EQNM,0.115180,0.099555,0.015625,0.0,0.0,1483.418318
3,2.0,30.0,6.0,5.0,1.0,1.0,1.0,0.0,1.0,-0.0,...,28461.14,4.0,739364,CHES,0.117072,0.107707,0.009365,0.0,0.0,266.533599
4,2.0,30.0,3.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,...,200426.79,4.0,739364,DGXX,0.160346,0.436161,-0.275816,0.0,0.0,-55280.828626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2.0,30.0,3.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,...,96779.37,4.0,739364,CFOI,0.160346,0.115393,0.044952,0.0,0.0,4350.454018
727,3.0,30.0,5.0,5.0,1.0,1.0,0.0,1.0,1.0,1.0,...,13911.56,4.0,739364,FHVG,0.121302,0.174542,-0.053240,0.0,0.0,-740.650040
728,3.0,30.0,4.0,3.0,1.0,1.0,0.0,1.0,1.0,0.0,...,92571.09,3.0,739364,ANFX,0.115233,0.084581,0.030651,0.0,0.0,2837.425492
729,3.0,20.0,5.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,...,489619.71,8.0,739364,EMGH,0.122228,0.026442,0.095786,0.0,0.0,46898.726854


In [18]:
final_1.to_csv(f"Optimized_result_Arun_Model_scipy_{zone}_{category}.csv", index = False)

In [29]:
forecast_with_efficiency = pd.concat([forecast.reset_index(drop = True), final_1['Initial_efficiency'].reset_index(drop = True)], axis = 1)

In [32]:
forecast_with_efficiency.drop("efficiency", axis = 1).to_csv(f"initial_x_{zone}_{category}.csv")